In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('2015_trip_data.csv')
startTime = pd.DatetimeIndex(data['starttime'])
data['dayOfWeek'] = startTime.dayofweek
data['date'] = startTime.date

In [ ]:
# Data cleaning
df_list = []
for i in range(7):
    data_DOW = data[startTime.dayofweek == i]
    from_counts = pd.value_counts(data_DOW.from_station_id)
    to_counts = pd.value_counts(data_DOW.to_station_id)
    df_counts = pd.DataFrame({'From': from_counts.sort_index(), 'To': to_counts.sort_index()})
    df_counts['weekday' + str(i)] = (df_counts['From'] - df_counts['To']) / pd.unique(data_DOW.date).shape[0]
    df_list.append(df_counts['weekday' + str(i)])   
df_combined = pd.concat(df_list,axis=1)
df_combined.columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
def plot_bar1(df, column, opts):
    """
    Does a bar plot for a single column.
    :param pd.DataFrame df:
    :param str column: name of the column to plot
    :param dict opts: key is plot attribute
    """
    n_groups = len(df.index)
    index = np.arange(n_groups)  # The "raw" x-axis of the bar plot
    rects1 = plt.bar(index, df[column])
    if 'xlabel' in opts:
        plt.xlabel(opts['xlabel'])
    if 'ylabel' in opts:
        plt.ylabel(opts['ylabel'])
    if 'xticks' in opts and opts['xticks']:
        plt.xticks(index, df.index)  # Convert "raw" x-axis into labels
        _, labels = plt.xticks()  # Get the new labels of the plot
        plt.setp(labels, rotation=90)  # Rotate labels to make them readable
    else:
        labels = ['' for x in df.index]
        plt.xticks(index, labels)   
    if 'ylim' in opts:
        plt.ylim(opts['ylim'])
    if 'title' in opts:
        plt.title(opts['title'])

In [ ]:
def plot_barN(df, columns, opts):
    """
    Does a bar plot for a single column.
    :param pd.DataFrame df:
    :param list-of-str columns: names of the column to plot
    :param dict opts: key is plot attribute
    """
    num_columns = len(columns)
    local_opts = dict(opts)  # Make a deep copy of the object
    idx = 0
    for column in columns:
        idx += 1
        local_opts['xticks'] = False
        local_opts['xlabel'] = ''
        if idx == num_columns:
            local_opts['xticks'] = True
            local_opts['xlabel'] = opts['xlabel']
        local_opts['title'] = column
        plt.subplot(num_columns, 1, idx)
        plot_bar1(df, column, local_opts)

In [ ]:
%matplotlib inline
import seaborn as sb
sb.set()
fig = plt.figure(figsize=(12, 10))  # Controls global properties of the bar plot
opts = {'xlabel': 'Stations', 'ylabel': 'Differences'}
plot_barN(df_combined, ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], opts)